In [3]:
# import 
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

torch.cuda.manual_seed_all(777)

In [5]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root = 'data/train_data',
                                              transform = trans)

data_loader = DataLoader(dataset = train_data,
                         batch_size = 8,
                         shuffle = True)

In [40]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Linear(16 * 13 * 29, 120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [44]:
# testing
model = CNN()

test_input = (list(data_loader)[0][0])
test_out = model(test_input)
test_out.shape

torch.Size([8, 2])

In [48]:
# cost/loss
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_func = nn.CrossEntropyLoss()

In [63]:
# training
total_batch = len(data_loader)

epochs = 5
for epoch in range(epochs):
    avg_cost = 0.0
    
    for idx, data in enumerate(data_loader):
        imgs, labels = data
        
        optimizer.zero_grad()
        prediction = model(imgs)
        cost = loss_func(prediction, labels - 1)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch:{}] cost = {:.>5}'.format(epoch+1, avg_cost))

[Epoch:1] cost = 0.0017562428256496787
[Epoch:2] cost = 0.0005073092761449516
[Epoch:3] cost = 0.0002804173855111003
[Epoch:4] cost = 0.00017398329509887844
[Epoch:5] cost = 0.00012286723358556628


In [70]:
torch.save(model.state_dict(), 'model/model.pth')

In [72]:
new_model = CNN()
new_model.load_state_dict(torch.load('model/model.pth'))

<All keys matched successfully>

In [75]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64, 128)),
    transforms.ToTensor()
])

test_data = torchvision.datasets.ImageFolder(root = 'data/test_data',
                                             transform = trans)

test_set = DataLoader(dataset = test_data,
                      batch_size = len(test_data))

In [78]:
with torch.no_grad():
    for idx, data in enumerate(test_set):
        imgs, labels = data
        
        prediction = new_model(imgs)
        correct_prediction = torch.argmax(prediction, 1) == labels
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy : ', accuracy.item())

Accuracy :  1.0
